In [1]:
#-- Modules & Packages

import yfinance as yf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import python_ta as ta

from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

In [ ]:
ticker = ['ETH-USD']

#-- Collecting 3 years
start_date = date.today() + relativedelta(months = - 60)
end_date = date.today()

#-- 1D ohlcv data collection
df_asset = yf.download(ticker, start_date, end_date, interval = '1D')
